# SMO

上面求对偶问题的时候我们说到SMO算法,但是没有具体说明,现在我们来看看.

$\underset{a}{max}(\sum_{i=1}^{n}a_i-\frac{1}{2}\sum_{i,j=1}^{n}a_ia_jy_iy_j<x_i,x_j>)$

s.t., $0\leqslant a_i \leqslant C,i=1,...,n$

$\sum_{i=1}^{n}a_iy_i=0$

等价于(这个等价不好推导建议直接知道结果去细想一下):

$\underset{a}{min}\Psi(\vec{a})=\underset{a}{min}\frac{1}{2}\sum_{i=1}^{N}\sum_{j=1}^{N}Y_iY_jK(\vec{x_i},\vec{x_j})a_ia_j-\sum_{i=1}^{N}a_i$

s.t., $0\leqslant a_i \leqslant C,\forall {i}$

$\sum_{i=1}^{N}a_iy_i=0$

1998年，Microsoft Research的John C. Platt在论文[《Sequential Minimal Optimization：A Fast Algorithm for Training Support Vector Machines》](https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/tr-98-14.pdf)中提出针对上述问题的解法：SMO算法，它很快便成为最快的二次规划优化算法，特别是在针对线性SVM和数据稀疏时性能更优。

#### 1.SMO算法推导

首先定义特征到结果的输出函数:

$u=\vec{W}\cdot\vec{X}-b$

这个u与我们之前定义的$f(x)=w^{T}x+b$是一样的

接着重新定义一下原始的优化问题:

$\underset{w,b}{min}\frac{1}{2}||\vec{w}||$ s.t. $y_i(\vec{w}\cdot\vec{x_i}-b) \geqslant1,\forall {i}$

求导得到:

$\vec{w}=\sum_{i=1}^{N}y_ia_i\vec{x_i},b=\vec{w}\cdot\vec{x_k}-y_k$ for som $a_k>0$

带入 $u=\vec{W}\cdot\vec{X}-b$得到,$u=\sum_{j=1}^{N}y_ja_jK(\vec{x_j},\vec{x})-b$

引入拉格朗日乘子转换后:

$\underset{a}{min}\Psi(\vec{a})=\underset{a}{min}\frac{1}{2}\sum_{i=1}^{N}\sum_{j=1}^{N}Y_iY_jK(\vec{x_i},\vec{x_j})a_ia_j-\sum_{i=1}^{N}a_i$

s.t., $ a_i \geqslant 0,\forall {i}$

$\sum_{i=1}^{N}a_iy_i=0$

加入松弛变量后,模型修改为:

$\underset{w,b,\xi}{min}\frac{1}{2}||\vec{w}||^{2}+C\sum_{i=1}^{N}$ s.t. $y_i(\vec{w}\cdot\vec{x_i}-b)\geqslant 1-\xi_i,\forall {i},0\leqslant a_i \leqslant C,\forall {i}$

最终我们的问题变成为:

$\underset{a}{min}\Psi(\vec{a})=\underset{a}{min}\frac{1}{2}\sum_{i=1}^{N}\sum_{j=1}^{N}Y_iY_jK(\vec{x_i},\vec{x_j})a_ia_j-\sum_{i=1}^{N}a_i$

s.t., $0\leqslant a_i \leqslant C,\forall {i}$

$\sum_{i=1}^{N}a_iy_i=0$

下面要解决的问题就是在上述$a_i=\{a_1,a_2,...,a_n\}$上求目标函数的最小值.那么我们按照之前的想法我们去固定$a_1$以外的所有参数,然后在$a_1$上去求极值,这样可行嘛?很显然是不行的,因为我们现在多了一个约束条件:

$\sum_{i=1}^{N}a_iy_i=0$

那么一旦固定$a_1$以外的所有参数,那么$a_1$也就固定下来了

$a_1y_1=-\sum_{i=2}^{N}a_iy_i$

所以我们现在需要固定除了$a_1,a_2$以外的所有参数这样的话$a_1,a_2$就不是死的了，也就是说目标函数只是关于$a_1,a_2$的函数,这样不断的在一堆乘子中去抽取两个求解,不断迭代,最终找到原问题的解.

因为我们已经固定了除了$a_1,a_2$,以外的所有乘子，所以除了$a_1,a_2$以外的所有乘子之和是一个常数,并且由于限制条件$\sum_{i=1}^{N}a_iy_i=0$的存在,那么形式如下:

$a_iy_i+a_2y_2 + \sum_{i=3}^{N}a_iy_i=0 \Rightarrow a_iy_i+a_2y_2 + \zeta =0$

$a_iy_i+a_2y_2=\zeta $

$\zeta $是一个常数

那么对于原对偶问题的子问题的目标函数可以表达为:

$\Psi=\frac{1}{2}K_{11}a_1^{2}+\frac{1}{2}K_{22}a_{2}^{2}+sK_{12}a_1a_2+y_1a_1v_1+y_2a_2v_2-a_2+\Psi_{constant} $

其中

$K_{ij}=K(\vec{x_i},\vec{x_j}),$

$v_i=\sum_{j=3}^{N}y_ja_jK_{ij}=u_i+b-y_1a_1K_{1i}-y_2a_2K_{2i}$